### API >> https://facebook-sdk.readthedocs.io/en/latest/api.html
### Get access token >> https://developers.facebook.com/tools/explorer

In [ ]:
!pip install facebook-sdk

In [1]:
#START
#this is my token (Pitágoras)
token = "EAACEdEose0cBAH2ihx79uCdJ2GdqTsDgBJAy1LJZBTKaTyg4w7Tl9nI3n43cIaJAHrEYK0jJ5QtWiIgTXieVJ56ze0872vYGKcsXZCi1aygDXUiJ3Fo3c6VLBa0OukO5pLZCW1RTjhAjkGzA2OxSDZACKdPxDc1LNrk2mzkeQ9fD0pjeHVOCH3zqgVxI3ysZD"
 
#ID of "Pitagoras"
otherPersonID = "10208935375967359"
raiID = "739425496128479"
anaID = "765831436837419"

In [2]:
from collections import defaultdict as ddict
import facebook as fb
import requests as req
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
#Gets the categories of pages liked by the user
#   token       user access token
#   return      dictionary of categories found, each with it's own array of pages
#       {c1 : [page1, page2, page3...], c2 : [page4, page5], c3 : [page6]...}
def getCats(token):
    cats = ddict(str)
    graph = fb.GraphAPI(access_token=token, version='2.7')
    resource = graph.get_object("me/likes?fields=name,category")
    #print(resource)
    cats = dict()
    while(True):
        for page in resource['data']:
            cat = page['category']
            if not(cat in cats):
                cats[cat] = []
            cats[cat].append(page['id'])
        # Attempt to make a request to the next page of data, if it exists.
        try:
            resource=req.get(resource['paging']['next']).json()
        except KeyError:
            #print("Finished getting pages")
            # When there are no more pages (['paging']['next']), break from the
            # loop and end the script.
            break
    return cats

In [4]:
#Classifies pages by categories, the category with more pages gets 1.0
#and the smaller categories get smaller and smaller classifications...
#   cats        dictionary of categories found, each with it's own array of pages
#       {c1 : [page1, page2, page3...], c2 : [page4, page5], c3 : [page6]...}
#   return      dictionary of pages, each with it's own classification (0.0 < classification <= 1.0)
#       {page1 : x, page2 : x, page4 : y, page6 : z}
def classify(cats):
    classification = dict()
    biggestCat = ""
    biggestCatLen = 0
    for cat, pages in cats.items():
        if(len(pages) > biggestCatLen):
            biggestCat = cat
            biggestCatLen = len(pages)
    for cat, pages in cats.items():
        classify = len(pages)/biggestCatLen
        for i in pages:
            classification[i] = classify

    return classification

In [12]:
import operator
normalWeigth = [25, 18, 15, 12, 10, 8, 6, 4, 2, 1]

# Normalizes the value previously attributed to each category of page
# Acording to the array 'normalWeigth', the category with the highest value
# receives a value of 25, the 2º 18, the 3º 15 and soo on... starting from the 10º,
# all categories receive a value of 1.
# c        dictionary of categories, each with its given value
# return   dictionary of categories, normalized
def norm(c):
    tuples = list()
    sortedByVal = sorted(c.items(), key=operator.itemgetter(1))
    
    for i in range(0, len(sortedByVal)-11):
        tup = sortedByVal[i]
        tuples.append((tup[0], 1))
    count = 9
    for i in range(len(sortedByVal)-9, len(sortedByVal)):
        a = normalWeigth[count]
        if(i > 0):
            tup = sortedByVal[i]
            tuples.append((tup[0], a))
            count = count - 1
    newDict = dict()
    for t in tuples:
        newDict[t[0]] = t[1]
    return newDict

In [6]:
#Get the pages that a given user liked
#   user    id of a user
#   token   access token
#   return  array of page IDs
#       [page1, page2, page3...]
def getPagesFromUser(user, token):
    pages = list()
    try:
        graph = fb.GraphAPI(access_token=token, version='2.7')
        resource = graph.get_object(user+"?fields=likes{category,name}")["likes"]
    except:
        return pages
    cats = dict()
    while(True):
        for page in resource['data']:
            pages.append(page['id'])
        # Attempt to make a request to the next page of data, if it exists.
        try:
            resource=req.get(resource['paging']['next']).json()
        except KeyError:
            #print("Finished getting pages from user " + user)
            # When there are no more pages (['paging']['next']), break from the
            # loop and end the script.
            break
    return pages

In [7]:
#Calculates the similarity between a user 
#and the current user we are searching friends for
#user              The ID of the other user
#classification    The classification of each page 
#                  the current user likes
#token             Current user access token
#return            The similarity rate, in float
def similarityWith(user, classification, token):
    otherUserPages = getPagesFromUser(user, token)
    rate = 0.0
    for page in otherUserPages:
        if page in classification:
            rate += classification[page]
    return rate

In [8]:
#Get only the friends of the user
#user     The user to search friends
#token    Access token
#return   The user's friends:
#            dict { id1 : friend_name1, id2 : friend_name2 }
def getFriends(user, token):
    graph = fb.GraphAPI(access_token=token, version='2.7')
    resource = graph.get_object("/" + user + "/friends")
    friends = dict()
    while(True):
        for friend in resource['data']:
            friends[friend['id']] = friend['name']
            # Attempt to make a request to the next page of data, if it exists.
        try:
            resource=req.get(resource['paging']['next']).json()
        except KeyError:
            #print("Finished getting friends from user " + user)
            # When there are no more pages (['paging']['next']), break from the
            # loop and end the script.
            break

    return friends

#Get the friends of the user and friends of friends too
#user     The user where the search for users starts
#token    Access token
#return   A dict of users:
#            dict { id1 : friend_name1, id2 : friend_name2 }
def getUsersDict(user, token):
    friends = getFriends(user, token)
    ffriends = dict() 
    listFFriends = []
    for idFriend in friends: 
        friendsOfFriends = getFriends(idFriend, token)
        for idUser, name in friendsOfFriends.items():
            if not idUser in ffriends and not idUser in friends:
                ffriends[idUser] = name
                listFFriends.append([idUser, name])
    
    count = 0
    while len(listFFriends) < 400 and len(listFFriends) > count:
        curruentID = listFFriends[count][0]
        friendsCurrentID = getFriends(curruentID, token)
        for idUser, name in friendsOfFriends.items():
            if not idUser in ffriends:
                ffriends[idUser] = name
                listFFriends.append([idUser, name])
        count+=1
    return ffriends

In [15]:
import threading
threadsLimit = 32
lock = threading.Lock()

#This function is supposed to be used as a thread to calculate users similarities
#When started, informs the counter that a new thread is running, and at the end,
#informs the counter that its no longer running.
#It uses a lock for safe concurrency.
#When the similarity ratio has been calculated, stores it on similarityDict
def similarityCalcThread(key, classf, token, similarityDict, threadsRunning):
    lock.acquire()
    threadsRunning = threadsRunning + 1
    lock.release()
    
    ratio = similarityWith(key, classf, token)
    similarityDict[key] = ratio
    
    lock.acquire()
    threadsRunning = threadsRunning - 1
    lock.release()

#Uses paralell processing to calculate the similarity of many users at the same time
#Waits for all the similarity's calculating threads to end
#Has a limit of running threads
def getSimilarityDict(friends, classf, token):
    similarityDict = dict()
    threadsRunning = 0
    threads = list()
    for key, value in friends.items():
        while(threadsRunning > threadsLimit):
            i = 1
            #do nothing
        t = threading.Thread(target=similarityCalcThread,
                             args = (key, classf, token, similarityDict, threadsRunning))
        t.daemon = False
        threads.append(t)
        t.start()
    
    #make sure ALL threads have ended
    for x in threads:
         x.join()
    return similarityDict

# Time to use the functions created above to do cool stuff
### The major problem with our proposal was that, since version 2.0, the Graph API only lets you access friend's likes. This eliminates the possibility of using like's similarity to compare two users that are not friends.
### So, for demonstration porpuses, we are only classifying the friends of the user.

In [10]:
users = getFriends('me', token)
#users = getUsersDict('me', token)

### We get the page categories, classify and normalize them

In [13]:
cats = getCats(token)
classf = norm(classify(cats))

### Now its time for the heavy processing, calculating the similarity with the other users

In [16]:
simDict = getSimilarityDict(users, classf, token)

# Pandas time! We got all the data, let's build the DataFrame and sort it

In [17]:
import pandas as pd

#Making a series out of the similarity dict
s = pd.Series(simDict, name='similarity')
s.index.name = 'user-id'
s.reset_index()
#Turn it into a dataframe and sort it
df_similarity = s.to_frame()
sortedDF = df_similarity.sort_values(["similarity"], ascending=[False])

#Now, append the user names, because we are humans
usersSeries = pd.Series(users, name='user-name')
usersSeries.index.name = 'user-id'
usersSeries.reset_index()
usersDF = usersSeries.to_frame()
sortedDF['user-name'] = usersDF['user-name']

#The final step
print (sortedDF)

                   similarity           user-name
user-id                                          
765831436837419          71.0        Ana Caroline
998100796886441          57.0         Ikaro Souza
739425496128479          42.0           Rai Vitor
773541116034720          23.0        Murilo Bento
862034200484130          22.0        Ivan Alisson
100002541369604          13.0  Aryan Dantas Gomes
100002211536120          12.0       Thuize Thainá
967446623299200           8.0         Paula Lopes
644347655699194           7.0       Kamilla Sâmia
653306944773428           5.0      Danilo Martins
10208935375967359         3.0     Gilberto Soares
100000888984231           0.0        Luan Fonseca
607827396001201           0.0     Inacio Medeiros
